# Review Outputs and Construct Table for interactive viz

In [1]:
library(sf)
library(tidyverse)
library(janitor)
library(magrittr)
library(lubridate)
library(fs)
library(glue)
library(readxl)
library(terra)
library(furrr)

# library(mapview)
# library(spatialEco)
# library(gfcanalysis)
outPath <- path('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm')
dir_create(outPath)

Linking to GEOS 3.11.1, GDAL 3.6.1, PROJ 9.1.0; sf_use_s2() is TRUE

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: timechange


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


terra 1.5.21


Attaching package: ‘terra’


The following object is masked from ‘packa

In [2]:
secondPath <- sf_proj_search_paths()[[2]]
secondPath
sf_proj_search_paths(secondPath)

[1] "/home/mmacande/.conda/envs/r4-base-20230102/share/proj"

[1] "/home/mmacande/.conda/envs/r4-base-20230102/share/proj"

In [3]:
srlite_fp <- read_sf('/explore/nobackup/people/pmontesa/srlite/footprints_vhrmeta_SR_alaska.gpkg') %>%
  mutate(srlite_path = path(path, file))
nrow(srlite_fp)

[1] 2963

In [4]:
ak_yukon_boreal_arctic <- read_sf('/explore/nobackup/people/mmacande/srlite/chm_model/gis_layers/AKVEG_Regions.gpkg', layer='NorthAmericanBeringia_ModelArea_3338') %>%
st_transform(4326)
ak_yukon_boreal_arctic %>% colnames()

# srlite_fp_ab <- st_filter(srlite_fp, ak_yukon_boreal_arctic)

srlite_fp <- srlite_fp %>%
mutate(arcBor = lengths(st_intersects(geom, ak_yukon_boreal_arctic)) > 0)

srlite_fp %>% st_drop_geometry() %>% group_by(arcBor) %>% tally()

[1] "Region"       "Shape_Length" "Shape_Area"   "geom"

st_as_s2(): dropping Z and/or M coordinate



arcBor,n
<lgl>,<int>
FALSE,506
TRUE,2457


In [5]:
srlite_summer <- srlite_fp %>% st_drop_geometry() %>% filter(month %in% c(6,7,8,9))
srlite_summer %>% group_by(month) %>% tally()
srlite_summer %>% head()

month,n
<dbl>,<int>
6,705
7,649
8,517
9,440


file,footprint_name,path,area_km2,area_ha,subdir,type,catid,sensor,year,month,date,satid,el,az,sunel,sunaz,srlite_path,arcBor
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fs::path>,<lgl>
WV02_20130813_M1BS_1030010026D4A600-sr-02m.tif,WV02_20130813_M1BS_1030010026D4A600,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_3,2629.7645,262976.45,/batch_1/4_band/split_3,SR,1030010026D4A600,WV02,2013,8,2013-08-13 01:00:00,WV02,74.70000,219.39,46.04000,175.1000,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_3/WV02_20130813_M1BS_1030010026D4A600-sr-02m.tif,FALSE
WV02_20170928_M1BS_103001007388B900-sr-02m.tif,WV02_20170928_M1BS_103001007388B900,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_6,3238.1923,323819.23,/batch_1/4_band/split_6,SR,103001007388B900,WV02,2017,9,2017-09-28 01:00:00,WV02,59.34000,258.02,23.11000,187.4800,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_6/WV02_20170928_M1BS_103001007388B900-sr-02m.tif,TRUE
WV03_20190704_M1BS_1040010050235800-sr-02m.tif,WV03_20190704_M1BS_1040010050235800,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2,2298.7670,229876.70,/batch_1/8_band_csv/split_2,SR,1040010050235800,WV03,2019,7,2019-07-04 01:00:00,WV03,66.00000,74.82,56.44000,160.3500,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2/WV03_20190704_M1BS_1040010050235800-sr-02m.tif,FALSE
WV03_20190806_M1BS_104001004E03D100-sr-02m.tif,WV03_20190806_M1BS_104001004E03D100,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2,1061.6519,106165.19,/batch_1/8_band_csv/split_2,SR,104001004E03D100,WV03,2019,8,2019-08-06 01:00:00,WV03,71.76667,266.60,50.25000,172.0667,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2/WV03_20190806_M1BS_104001004E03D100-sr-02m.tif,FALSE
WV03_20190830_M1BS_104001004FD68A00-sr-02m.tif,WV03_20190830_M1BS_104001004FD68A00,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2,411.4221,41142.21,/batch_1/8_band_csv/split_2,SR,104001004FD68A00,WV03,2019,8,2019-08-30 01:00:00,WV03,81.60000,74.70,42.36667,169.0000,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2/WV03_20190830_M1BS_104001004FD68A00-sr-02m.tif,FALSE
WV03_20190830_M1BS_1040010051483500-sr-02m.tif,WV03_20190830_M1BS_1040010051483500,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2,789.5285,78952.85,/batch_1/8_band_csv/split_2,SR,1040010051483500,WV03,2019,8,2019-08-30 01:00:00,WV03,73.24000,165.78,42.28000,168.9400,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_2/WV03_20190830_M1BS_1040010051483500-sr-02m.tif,FALSE


In [6]:
srlite_metrics <- read_csv('/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/tables/srlite_metrics_alaska_202308291350.csv') %>%
  select(-`...1`)
dups <- srlite_metrics %>%
  dplyr::group_by(model, file, type, footprint_name, catid, sensor, year, month, date, band_name) %>%
  dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
  dplyr::filter(n > 1L) 


srlite_metrics_wide <- srlite_metrics %>%
  filter(band_name %in% c('Blue','Green','Red','NIR')) %>%
  pivot_wider(id_cols=c(model, file, type, footprint_name, catid, sensor, year, month, date), names_from=band_name, values_from=intercept:rmse_norm, values_fn=mean) #%>%

srlite_metrics_wide_78 <- srlite_metrics_wide %>%
  filter(month %in% c(7,8))

srlite_metrics_wide_78_r2filt <- srlite_metrics_wide_78 %>%
  filter(r2_score_NIR >= 0.75)

srlite_metrics_wide_r2filt <- srlite_metrics_wide %>%
  filter(r2_score_NIR >= 0.75)

nrow(srlite_metrics_wide_78_r2filt)

srlite_metrics_wide_r2filt %>% head()

New names:
• `` -> `...1`
Rows: 25160 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (7): band_name, model, file, type, footprint_name, catid, sensor
dbl  (17): ...1, intercept, slope, r2_score, explained_variance, mbe, mae, m...
date  (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 797

model,file,type,footprint_name,catid,sensor,year,month,date,intercept_Blue,⋯,mean_evhr_srlite_Red,mean_evhr_srlite_NIR,mae_norm_Blue,mae_norm_Green,mae_norm_Red,mae_norm_NIR,rmse_norm_Blue,rmse_norm_Green,rmse_norm_Red,rmse_norm_NIR
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
rma,WV02_20100526_M1BS_1030010005890800_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100526_M1BS_1030010005890800,1030010005890800,WV02,2010,5,2010-05-26,-0.004624824,⋯,0.03532135,0.1458139,0.20889571,0.24253339,0.31376809,0.12884627,0.3837424,0.41164465,0.5566762,0.21242917
rma,WV02_20100607_M1BS_10300100056B1D00_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100607_M1BS_10300100056B1D00,10300100056B1D00,WV02,2010,6,2010-06-07,-0.060267183,⋯,0.05153005,0.1790403,0.06466824,0.05240292,0.07740959,0.06252787,0.1078076,0.09183965,0.1347577,0.09222752
rma,WV02_20100624_M1BS_10300100065B9F00_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100624_M1BS_10300100065B9F00,10300100065B9F00,WV02,2010,6,2010-06-24,-0.082692242,⋯,0.07537508,0.1878855,0.12921003,0.10444712,0.10986816,0.06081597,0.3594432,0.29748031,0.2911994,0.11510198
rma,WV02_20100627_M1BS_103001000625B000_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100627_M1BS_103001000625B000,103001000625B000,WV02,2010,6,2010-06-27,0.111568724,⋯,0.05883414,0.1838538,0.26085863,0.36552352,0.41956373,0.11296184,0.3298210,0.44748244,0.5191334,0.15892230
rma,WV02_20100707_M1BS_1030010005554F00_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100707_M1BS_1030010005554F00,1030010005554F00,WV02,2010,7,2010-07-07,-0.026643385,⋯,0.04253981,0.2035741,0.28470133,0.24084641,0.32272155,0.14984594,0.7463661,0.52428886,0.6934603,0.23713079
rma,WV02_20100709_M1BS_1030010006544A00_rma_SRLite_statistics.csv,SR (batch_1 4_band_csv),WV02_20100709_M1BS_1030010006544A00,1030010006544A00,WV02,2010,7,2010-07-09,-0.048322465,⋯,0.03969153,0.2493493,0.09636465,0.07176906,0.09862164,0.06099099,0.1439473,0.10725108,0.1486887,0.08519820


In [7]:
srlite_summer_r2_filt <- srlite_summer %>%
semi_join(srlite_metrics_wide_r2filt, by='catid')

# srlite_summer %>% head()
# srlite_metrics_wide_r2filt %>% head()

srlite_summer_r2_filt %>% group_by(month) %>% tally()

month,n
<dbl>,<int>
6,348
7,413
8,324
9,276


In [8]:
predicted_set1 <- read_csv('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/list_srlite_path_for_predstack_v20230829.csv') %>%
mutate(srlite_file = path_file(srlite_path),
       catid = str_sub(srlite_file,20,35))

predicted_set1 %>% head()

srlite_batch2 <- srlite_summer_r2_filt %>%
anti_join(predicted_set1)

srlite_batch2 %>% group_by(month, arcBor) %>% tally()
# srlite_batch2

Rows: 340 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): srlite_path

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


srlite_path,srlite_file,catid
<chr>,<chr>,<chr>
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100802_M1BS_1030010006545100-sr-02m.tif,WV02_20100802_M1BS_1030010006545100-sr-02m.tif,1030010006545100
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100815_M1BS_10300100068BF600-sr-02m.tif,WV02_20100815_M1BS_10300100068BF600-sr-02m.tif,10300100068BF600
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110707_M1BS_103001000C6D7600-sr-02m.tif,WV02_20110707_M1BS_103001000C6D7600-sr-02m.tif,103001000C6D7600
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110707_M1BS_103001000CA39000-sr-02m.tif,WV02_20110707_M1BS_103001000CA39000-sr-02m.tif,103001000CA39000
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110721_M1BS_103001000C603400-sr-02m.tif,WV02_20110721_M1BS_103001000C603400-sr-02m.tif,103001000C603400
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110722_M1BS_103001000CB60600-sr-02m.tif,WV02_20110722_M1BS_103001000CB60600-sr-02m.tif,103001000CB60600


Joining, by = c("catid", "srlite_path")


month,arcBor,n
<dbl>,<lgl>,<int>
6,FALSE,46
6,TRUE,302
7,FALSE,78
7,TRUE,131
8,FALSE,79
8,TRUE,109
9,FALSE,92
9,TRUE,184


In [18]:
#Only run when data is updated
if (F) {
write_csv(srlite_batch2 %>% filter(month %in% c(7,8), arcBor) %>% select(srlite_path), '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/list_batch2_julyAug_srlite_path_for_predstack_v20230915.csv')
write_csv(srlite_batch2 %>% filter(month %in% c(6,9), arcBor) %>% select(srlite_path), '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/list_batch3_juneSept_srlite_path_for_predstack_v20230915.csv')
write_csv(srlite_batch2 %>% filter(!arcBor) %>% select(footprint_name), '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/list_batch4_JJAS_SEAK_srlite_path_toDelete_v20230915.csv')
}

# Generate table of CHMs for viewer

In [9]:
cloudmasks <- tibble(cloudmask_path = dir_ls('/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask', glob='*/*cloudmask.tif', recurse=T)) %>%
mutate(cloudmask_file = path_file(cloudmask_path),
       catid = str_sub(cloudmask_file, 20, 35))
nrow(cloudmasks)
cloudmasks %>% head()

[1] 3297

cloudmask_path,cloudmask_file,catid
<fs::path>,<chr>,<chr>
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100502_M1BS_10300100045FA300-toa.cloudmask.tif,WV02_20100502_M1BS_10300100045FA300-toa.cloudmask.tif,10300100045FA300
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100505_M1BS_1030010005215900-toa.cloudmask.tif,WV02_20100505_M1BS_1030010005215900-toa.cloudmask.tif,1030010005215900
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100526_M1BS_1030010005890800-toa.cloudmask.tif,WV02_20100526_M1BS_1030010005890800-toa.cloudmask.tif,1030010005890800
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100529_M1BS_10300100054FCA00-toa.cloudmask.tif,WV02_20100529_M1BS_10300100054FCA00-toa.cloudmask.tif,10300100054FCA00
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100531_M1BS_10300100053C1900-toa.cloudmask.tif,WV02_20100531_M1BS_10300100053C1900-toa.cloudmask.tif,10300100053C1900
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100607_M1BS_10300100056B1D00-toa.cloudmask.tif,WV02_20100607_M1BS_10300100056B1D00-toa.cloudmask.tif,10300100056B1D00


In [10]:
# ccdc <- tibble(ccdc_path = dir_ls('/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/ccdc', glob='*/*ccdc.tif', recurse=T)) %>%
ccdc <- bind_rows(
    tibble(ccdc_path = dir_ls('/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/', glob='*/*ccdc.tif', recurse=T),
           source = '/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska'),
    tibble(ccdc_path = dir_ls('/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230807_alaska_batch23/alaska', glob='*/*ccdc.tif', recurse=T),
           source = '/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230807_alaska_batch23') ) %>%
mutate(ccdc_file = path_file(ccdc_path),
       catid = str_sub(ccdc_file, 20, 35))

nrow(ccdc)
# ccdc %>% head()
dups <- ccdc %>% group_by(catid) %>% tally() %>% filter(n > 1)
print('duplicates:')
nrow(dups)
      
ccdc <- bind_rows(
    ccdc %>% filter(source == '/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska') %>% anti_join(dups),
    ccdc %>% filter(source == '/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230807_alaska_batch23'))
nrow(ccdc)


[1] 4993

[1] "duplicates:"


[1] 92

Joining, by = "catid"


[1] 4901

In [11]:
toas <- tibble(toa_path = dir_ls('/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa', glob='*/*toa.tif', recurse=T)) %>%
mutate(toa_file = path_file(toa_path),
       catid = str_sub(toa_file, 20, 35))

nrow(toas)
toas %>% head()

[1] 3343

toa_path,toa_file,catid
<fs::path>,<chr>,<chr>
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100502_M1BS_10300100045FA300-toa.tif,WV02_20100502_M1BS_10300100045FA300-toa.tif,10300100045FA300
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100505_M1BS_1030010005215900-toa.tif,WV02_20100505_M1BS_1030010005215900-toa.tif,1030010005215900
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100526_M1BS_1030010005890800-toa.tif,WV02_20100526_M1BS_1030010005890800-toa.tif,1030010005890800
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100529_M1BS_10300100054FCA00-toa.tif,WV02_20100529_M1BS_10300100054FCA00-toa.tif,10300100054FCA00
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100531_M1BS_10300100053C1900-toa.tif,WV02_20100531_M1BS_10300100053C1900-toa.tif,10300100053C1900
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100607_M1BS_10300100056B1D00-toa.tif,WV02_20100607_M1BS_10300100056B1D00-toa.tif,10300100056B1D00


In [12]:
chms_predicted <- tibble(
    chm_path = dir_ls('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products', glob='*/*.tif', recurse=T)) %>%
    mutate(chm_file = path_file(chm_path),
           catid = str_sub(chm_file,20,35)) %>%
select(catid, chm_file, chm_path) %>%
left_join(srlite_fp %>% select(catid, srlite_path, arcBor)) %>%
left_join(cloudmasks %>% select(catid, cloudmask_path)) %>%
left_join(toas %>% select(catid, toa_path)) %>%
left_join(ccdc %>% select(catid, ccdc_path)) %>%
st_as_sf()
 
nrow(chms_predicted)
head(chms_predicted %>% st_drop_geometry())
write_csv(chms_predicted %>% st_drop_geometry(), '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_20230915.csv')
write_sf(chms_predicted, '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_20230915.gpkg')

Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"


[1] 340

catid,chm_file,chm_path,srlite_path,arcBor,cloudmask_path,toa_path,ccdc_path
<chr>,<chr>,<fs::path>,<fs::path>,<lgl>,<fs::path>,<fs::path>,<fs::path>
1030010006545100,WV02_20100802_M1BS_1030010006545100-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20100802_M1BS_1030010006545100-sr-02m/WV02_20100802_M1BS_1030010006545100-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100802_M1BS_1030010006545100-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100802_M1BS_1030010006545100-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100802_M1BS_1030010006545100-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100802_M1BS_1030010006545100-ccdc.tif
103001000617C000,WV02_20100804_M1BS_103001000617C000-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20100804_M1BS_103001000617C000-sr-02m/WV02_20100804_M1BS_103001000617C000-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_4/WV02_20100804_M1BS_103001000617C000-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_4/WV02_20100804_M1BS_103001000617C000-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/split_4/WV02_20100804_M1BS_103001000617C000-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100804_M1BS_103001000617C000-ccdc.tif
10300100068BF600,WV02_20100815_M1BS_10300100068BF600-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20100815_M1BS_10300100068BF600-sr-02m/WV02_20100815_M1BS_10300100068BF600-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100815_M1BS_10300100068BF600-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100815_M1BS_10300100068BF600-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100815_M1BS_10300100068BF600-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100815_M1BS_10300100068BF600-ccdc.tif
103001000C6D7600,WV02_20110707_M1BS_103001000C6D7600-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20110707_M1BS_103001000C6D7600-sr-02m/WV02_20110707_M1BS_103001000C6D7600-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110707_M1BS_103001000C6D7600-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20110707_M1BS_103001000C6D7600-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20110707_M1BS_103001000C6D7600-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20110707_M1BS_103001000C6D7600-ccdc.tif
103001000CA39000,WV02_20110707_M1BS_103001000CA39000-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20110707_M1BS_103001000CA39000-sr-02m/WV02_20110707_M1BS_103001000CA39000-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110707_M1BS_103001000CA39000-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20110707_M1BS_103001000CA39000-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/

In [13]:
chms_predicted2 <- tibble(
    chm_path = dir_ls('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2', glob='*/*.tif', recurse=T)) %>%
    mutate(chm_file = path_file(chm_path),
           catid = str_sub(chm_file,20,35)) %>%
select(catid, chm_file, chm_path) %>%
left_join(srlite_fp %>% select(catid, srlite_path, arcBor)) %>%
left_join(cloudmasks %>% select(catid, cloudmask_path)) %>%
left_join(toas %>% select(catid, toa_path)) %>%
left_join(ccdc %>% select(catid, ccdc_path)) %>%
st_as_sf()

nrow(chms_predicted2)
head(chms_predicted2 %>% st_drop_geometry())
write_csv(chms_predicted2 %>% st_drop_geometry(), '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_batch2_20230915.csv')
write_sf(chms_predicted2, '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_batch2_20230915.gpkg')

Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"


[1] 239

catid,chm_file,chm_path,srlite_path,arcBor,cloudmask_path,toa_path,ccdc_path
<chr>,<chr>,<fs::path>,<fs::path>,<lgl>,<fs::path>,<fs::path>,<fs::path>
1030010006544A00,WV02_20100709_M1BS_1030010006544A00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2/WV02_20100709_M1BS_1030010006544A00-sr-02m/WV02_20100709_M1BS_1030010006544A00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100709_M1BS_1030010006544A00-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100709_M1BS_1030010006544A00-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100709_M1BS_1030010006544A00-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100709_M1BS_1030010006544A00-ccdc.tif
1030010006A83500,WV02_20100820_M1BS_1030010006A83500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2/WV02_20100820_M1BS_1030010006A83500-sr-02m/WV02_20100820_M1BS_1030010006A83500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100820_M1BS_1030010006A83500-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100820_M1BS_1030010006A83500-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100820_M1BS_1030010006A83500-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100820_M1BS_1030010006A83500-ccdc.tif
103001000C6B2700,WV02_20110718_M1BS_103001000C6B2700-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2/WV02_20110718_M1BS_103001000C6B2700-sr-02m/WV02_20110718_M1BS_103001000C6B2700-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110718_M1BS_103001000C6B2700-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20110718_M1BS_103001000C6B2700-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20110718_M1BS_103001000C6B2700-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20110718_M1BS_103001000C6B2700-ccdc.tif
103001000C734500,WV02_20110812_M1BS_103001000C734500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2/WV02_20110812_M1BS_103001000C734500-sr-02m/WV02_20110812_M1BS_103001000C734500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_6_0/WV02_20110812_M1BS_103001000C734500-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_6/WV02_20110812_M1BS_103001000C734500-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/split_6_0/WV02_20110812_M1BS_103001000C734500-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20110812_M1BS_103001000C734500-ccdc.tif
103001000C7EBF00,WV02_20110812_M1BS_103001000C7EBF00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products_batch2/WV02_20110812_M1BS_103001000C7EBF00-sr-02m/WV02_20110812_M1BS_103001000C7EBF00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_5/WV02_20110812_M1BS_103001000C7EBF00-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_5/WV02_20110812_M1BS_103001000C7EBF00-toa.cloudmask.tif,/explore/nobackup/projects/ila

In [15]:
chms_predicted_20230916 <- tibble(
    chm_path = dir_ls('/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2', glob='*/*.tif', recurse=T)) %>%
    mutate(chm_file = path_file(chm_path),
           catid = str_sub(chm_file,20,35)) %>%
select(catid, chm_file, chm_path) %>%
left_join(srlite_fp %>% select(catid, srlite_path, arcBor)) %>%
left_join(cloudmasks %>% select(catid, cloudmask_path)) %>%
left_join(toas %>% select(catid, toa_path)) %>%
left_join(ccdc %>% select(catid, ccdc_path)) %>%
st_as_sf()

nrow(chms_predicted_20230916)
head(chms_predicted_20230916 %>% st_drop_geometry())
write_csv(chms_predicted_20230916 %>% st_drop_geometry(), '/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/chm_srlite_list_20230918.csv')
write_sf(chms_predicted_20230916, '/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/chm_srlite_list_20230918.gpkg')

Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"
Joining, by = "catid"


[1] 204

catid,chm_file,chm_path,srlite_path,arcBor,cloudmask_path,toa_path,ccdc_path
<chr>,<chr>,<fs::path>,<fs::path>,<lgl>,<fs::path>,<fs::path>,<fs::path>
1030010006544A00,WV02_20100709_M1BS_1030010006544A00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2/WV02_20100709_M1BS_1030010006544A00-sr-02m/WV02_20100709_M1BS_1030010006544A00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100709_M1BS_1030010006544A00-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100709_M1BS_1030010006544A00-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100709_M1BS_1030010006544A00-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100709_M1BS_1030010006544A00-ccdc.tif
1030010006A83500,WV02_20100820_M1BS_1030010006A83500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2/WV02_20100820_M1BS_1030010006A83500-sr-02m/WV02_20100820_M1BS_1030010006A83500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20100820_M1BS_1030010006A83500-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20100820_M1BS_1030010006A83500-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20100820_M1BS_1030010006A83500-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20100820_M1BS_1030010006A83500-ccdc.tif
103001000C6B2700,WV02_20110718_M1BS_103001000C6B2700-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2/WV02_20110718_M1BS_103001000C6B2700-sr-02m/WV02_20110718_M1BS_103001000C6B2700-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band_csv/WV02_20110718_M1BS_103001000C6B2700-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/4_bands/WV02_20110718_M1BS_103001000C6B2700-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/4_bands/WV02_20110718_M1BS_103001000C6B2700-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20110718_M1BS_103001000C6B2700-ccdc.tif
103001000C734500,WV02_20110812_M1BS_103001000C734500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2/WV02_20110812_M1BS_103001000C734500-sr-02m/WV02_20110812_M1BS_103001000C734500-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_6_0/WV02_20110812_M1BS_103001000C734500-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_6/WV02_20110812_M1BS_103001000C734500-toa.cloudmask.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/split_6_0/WV02_20110812_M1BS_103001000C734500-toa.tif,/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20110812_M1BS_103001000C734500-ccdc.tif
103001000C7EBF00,WV02_20110812_M1BS_103001000C7EBF00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/products_batch2/WV02_20110812_M1BS_103001000C7EBF00-sr-02m/WV02_20110812_M1BS_103001000C7EBF00-sr-02m.cnn-chm-v1.tif,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_5/WV02_20110812_M1BS_103001000C7EBF00-sr-02m.tif,TRUE,/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_5/WV02_20110812_M1BS_103001000C7EBF00-toa.cloudmask.tif,/explore/nobackup/projects/ila